In [212]:
# Importing the necessary libraries
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import glob
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_fscore_support
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict

In [213]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 400)

In [214]:
df = pd.read_pickle("./Data/dataframefull.pickle")

In [215]:
df.head()

,visit_id,user_id,visit_number,date,day_of_week,article_name,article_id,price,quantity,total_price,shop_cart_value,shop_cart_item_count,shop_cart_unique_articles,country,region,city,city_id,women_article,men_article,kids_article,homeandliving_article,campaign_involved,category,category_slim,brand,product_line,product_group,product_line_r,product_group_r,genre_column_placeholder,device_category,device_browser,device_os,stock_value,year_month,min_purchase_date,user_type
0,551960553811,553811,1,2018-09-10,0,Geschenkgutschein 10 EUR,200000000010,10.0,1,10.0,30.0,2,2,Germany,Lower Saxony,Brunswick,1004478,0,1,0,0,False,Herren,men,(not set),Herren,Herren,Herren,Herren,Geschenkgutschein 10 EUR/Herren/Herren/Herren/...,desktop,Firefox,Windows,5.77,201809,201809,first
1,551960553811,553811,1,2018-09-10,0,Geschenkgutschein 20 EUR,200000000020,20.0,1,20.0,30.0,2,2,Germany,Lower Saxony,Brunswick,1004478,0,1,0,0,False,Herren,men,(not set),Herren,Herren,Herren,Herren,Geschenkgutschein 20 EUR/Herren/Herren/Herren/...,desktop,Firefox,Windows,5.77,201809,201809,first
2,242735254033,254033,1,2018-01-04,3,Geschenkgutschein 10 EUR,200000000010,10.0,1,10.0,80.0,3,3,Germany,Lower Saxony,Wunstorf,9048892,1,1,0,0,True,Damen/Geschenkgutscheine,women/men,(not set),Herren,Herren,Herren,Herren,Geschenkgutschein 10 EUR/Damen/Geschenkgutsche...,desktop,Firefox,Windows,11.40,201801,201801,first
3,242735254033,254033,1,2018-01-04,3,Geschenkgutschein 20 EUR,200000000020,20.0,1,20.0,80.0,3,3,Germany,Lower Saxony,Wunstorf,9048892,1,1,0,0,True,Damen/Geschenkgutscheine,women/men,(not set),Herren,Herren,Herren,Herren,Geschenkgutschein 20 EUR/Damen/Geschenkgutsche...,desktop,Firefox,Windows,11.40,201801,201801,first
4,242735254033,254033,1,2018-01-04,3,Geschenkgutschein 50 EUR,200000000050,50.0,1,50.0,80.0,3,3,Germany,Lower Saxony,Wunstorf,9048892,1,1,0,0,True,Damen/Geschenkgutscheine,women/men,(not set),Herren,Herren,Herren,Herren,Geschenkgutschein 50 EUR/Damen/Geschenkgutsche...,desktop,Firefox,Windows,11.40,201801,201801,first


## sorted by month

In [216]:
#total revenue per month
df_month = df.groupby(['year_month'])['total_price'].sum().reset_index()

In [217]:
#women articles per month:
df_month = df_month.join(df.groupby(['year_month'])['women_article'].sum().reset_index().drop(columns="year_month"))
#men articles per month:
df_month = df_month.join(df.groupby(['year_month'])['men_article'].sum().reset_index().drop(columns="year_month"))
#kids articles per month:
df_month = df_month.join(df.groupby(['year_month'])['kids_article'].sum().reset_index().drop(columns="year_month"))
#home and living articles per month:
df_month = df_month.join(df.groupby(['year_month'])['homeandliving_article'].sum().reset_index().drop(columns="year_month"))
#average shopping cart value per month:
df_month = df_month.join(df.drop_duplicates(subset ="visit_id").groupby(['year_month'])['shop_cart_value'].mean().reset_index().drop(columns="year_month"))
#number of active customers per month:
df_month = df_month.join(df.groupby('year_month')['user_id'].nunique().reset_index().drop(columns="year_month"))
#number of total purchases per month:
df_month = df_month.join(df.groupby('year_month')['visit_id'].nunique().reset_index().drop(columns="year_month"))
#average number of items bought per shopping cart:
df_month = df_month.join(df.drop_duplicates(subset ="visit_id").groupby(['year_month'])['shop_cart_item_count'].mean().reset_index().drop(columns="year_month"))



#rename columns to better reflect what they represent
df_month = df_month.rename(columns = {"total_price": "revenue", "shop_cart_value": "avg_shop_cart", "visit_id": "total_purchases", "user_id": "active_customers", "shop_cart_item_count": "avg_cart_itemcount"})

In [218]:
#add monthly growth of revenue:
df_month["monthly_growth_revenue"] = df_month["revenue"].pct_change()
#fill the one NaN that comes with the above code:
df_month = df_month.fillna(0)

In [219]:
#reordering the columns
cols = df_month.columns.tolist()
cols = cols[0:2] + [cols[10]] + [cols[6]] + [cols[9]] + cols[7:9] + cols[2:6] 
print("Check if any column was skipped when reordering (True: all columns fetched):", len(cols) == len(df_month.columns))
df_month = df_month.reindex(columns = cols)

Check if any column was skipped when reordering (True: all columns fetched): True


In [220]:
#revenue for first users
df_month = df_month.join(df[df["user_type"] == "first"].groupby(['year_month'])["total_price"].sum().reset_index().drop(columns="year_month"))
df_month = df_month.rename(columns = {"total_price": "revenue_first"})

In [221]:
#revenue for established users
row = pd.Series({201711: 0},name=0)
est_df = df[df["user_type"] == "established"].groupby(['year_month'])["total_price"].sum().append(row)
est_df = est_df.sort_index()
est_df = est_df.reset_index()
est_df = est_df.rename(columns = {"index": "year_month", 0: "revenue_established"})
df_month = df_month.join(est_df.drop(columns="year_month"))

In [222]:
#count of unique first users each month
df_month = df_month.join(df[df["user_type"] == "first"].groupby(['year_month'])["user_id"].nunique().reset_index().drop(columns="year_month"))
df_month = df_month.rename(columns = {"user_id": "first_user_count"})

In [223]:
#count for established users
row = pd.Series({201711: 0},name=0)
estc_df = df[df["user_type"] == "established"].groupby(['year_month'])["user_id"].nunique().append(row)
estc_df = estc_df.sort_index()
estc_df = estc_df.reset_index()
estc_df = estc_df.rename(columns = {"index": "year_month", 0: "established_user_count"})
df_month = df_month.join(estc_df.drop(columns="year_month"))

In [249]:
#revenue per month
u_purch = df.groupby(['user_id','year_month'])['total_price'].sum().reset_index()

#retention matrix with crosstab
retention = pd.crosstab(u_purch['user_id'], u_purch['year_month']).reset_index()

#retained and uder per month
months = retention.columns[:]
retention_array = []
for i in range(len(months)-1):
    retention_data = {}
    selected_month = months[i+1]
    prev_month = months[i]
    retention_data['year_month'] = int(selected_month)
    retention_data['user_count'] = retention[selected_month].sum()
    retention_data['retained_user_count'] = retention[(retention[selected_month]>0) & (retention[prev_month]>0)][selected_month].sum()
    retention_array.append(retention_data)
    
#array to dataframe and add retention_rate
retention = pd.DataFrame(retention_array)
retention['retention_rate'] = retention['retained_user_count']/retention['user_count']


In [252]:
df_month = df_month.join(retention.drop(columns = ["user_count", "year_month"]))

In [253]:
df_month

,year_month,revenue,monthly_growth,avg_shop_cart,avg_cart_itemcount,active_customers,total_purchases,women_article,men_article,kids_article,homeandliving_article,revenue_first,revenue_established,first_user_count,established_user_count,retained_user_count,retention_rate
0,201711,5.268517e+06,0.000000,103.495016,3.466743,46985,50906,96910,61491,14107,999,5.268517e+06,0.00,46985,0,46985,1.000000
1,201712,4.983451e+06,-0.054108,90.652698,3.284649,50004,54973,95202,64962,15461,1806,4.619641e+06,363809.47,46799,3205,3205,0.064095
2,201801,3.717325e+06,-0.254066,87.034361,3.661563,38221,42711,89365,52416,10983,1100,3.216473e+06,500851.97,33596,4625,3141,0.082180
3,201802,3.056483e+06,-0.177773,90.680675,3.791995,30837,33706,67535,42438,14910,795,2.526812e+06,529670.62,26114,4723,2520,0.081720
4,201803,6.210612e+06,1.031947,108.247849,3.966274,52209,57374,139458,66852,18202,964,5.122223e+06,1088389.01,44512,7697,2797,0.053573
5,201804,5.107636e+06,-0.177595,104.993942,3.836006,44879,48648,108200,62834,12806,654,4.138663e+06,968972.92,37316,7563,3795,0.084561
6,201805,6.152866e+06,0.204641,102.329341,4.169192,54426,60129,143037,88312,17691,674,4.803902e+06,1348963.69,44309,10117,3980,0.073127
7,201806,4.380521e+06,-0.288052,87.484437,3.756251,45145,50072,104008,71759,9499,1068,3.318716e+06,1061804.23,35922,9223,4214,0.093344
8,201807,4.071556e+06,-0.070532,75.919369,4.298135,48122,53630,132839,85224,10547,936,3.026853e+06,1044702.83,37665,10457,4201,0.087299
9,201808,3.455965e+06,-0.151193,80.592434,3.929901,38260,42882,98669,56584,12422,660,2.501735e+06,954229.67,29175,9085,3823,0.099922


In [ ]:
#TD eventuell growth of users

## sorted by user

In [224]:
#initialize dataframe with total revenue per customer
df_user = df.groupby('user_id')['total_price'].sum().reset_index()

In [225]:
#women articles the user bought:
df_user = df_user.join(df.groupby(['user_id'])['women_article'].sum().reset_index().drop(columns="user_id"))
#men articles the user bought:
df_user = df_user.join(df.groupby(['user_id'])['men_article'].sum().reset_index().drop(columns="user_id"))
#kids articles the user bought:
df_user = df_user.join(df.groupby(['user_id'])['kids_article'].sum().reset_index().drop(columns="user_id"))
#home and living articles the user bought:
df_user = df_user.join(df.groupby(['user_id'])['homeandliving_article'].sum().reset_index().drop(columns="user_id"))
#average shopping cart value per customer:
df_user = df_user.join(df.drop_duplicates(subset ="visit_id").groupby(['user_id'])['shop_cart_value'].mean().reset_index().drop(columns="user_id"))
#number of total visits per user:
df_user = df_user.join(df.groupby('user_id')['visit_id'].nunique().reset_index().drop(columns="user_id"))
#average number of items bought per shopping cart:
df_user = df_user.join(df.drop_duplicates(subset ="visit_id").groupby(['user_id'])['shop_cart_item_count'].mean().reset_index().drop(columns="user_id"))







#rename columns to better reflect what they represent
df_user = df_user.rename(columns = {"total_price": "revenue", "shop_cart_value": "avg_shop_cart", "visit_id": "total_purchases", "shop_cart_item_count": "avg_cart_itemcount"})

In [226]:
# entry for date + user for easier use
df["date_user"] = df['date'].map(lambda date: 10000*date.year + 100*date.month + date.day).apply(lambda x: str(x)) + df.user_id.apply(lambda x: str(x))

In [227]:
df.day_of_week.replace(0, 7, inplace=True)

In [228]:
def setday(day, daynr):
    """
    Set the days for users in the df_user dataframe
    day - weekday , Monday .... Sunday
    daynr - nr of day , here 7,1,2,...,6
    """
    uniq_d = df[df["day_of_week"] == daynr].drop_duplicates(subset = "date_user").groupby('user_id')["day_of_week"].sum().reset_index()
    uniq_d = uniq_d.set_index("user_id")
    del uniq_d.index.name
    uniq_calc = (uniq_d//daynr)
    uniq_dic = uniq_calc.to_dict()
    num_d = df[df["day_of_week"] == daynr].drop_duplicates(subset ="visit_id").groupby('user_id')["day_of_week"].sum().reset_index()
    num_d = num_d.set_index("user_id")
    num_d_calc = ((num_d - uniq_d)//daynr)+1  #articles per day
    num_dic = num_d_calc.to_dict()
    df_user[str(day)] = df_user["user_id"].map(uniq_dic["day_of_week"])
    df_user["times_bought_" + str(day)] = df_user["user_id"].map(num_dic["day_of_week"])

In [229]:
daylist = [("monday", 7), ("tuesday", 1), ("wednesday", 2), ("thursday", 3), ("friday", 4), ("saturday", 5), ("sunday", 6)]

In [230]:
for i, j in daylist:
    setday(i,j)

In [235]:
df_user = df_user.fillna(0)

In [237]:
#replace monday back to 0
df.day_of_week.replace(7, 0, inplace=True)

In [21]:
# visit , user , total_price, shopvalue, shopcount,articles, 

In [241]:
df_user.head()

,user_id,revenue,women_article,men_article,kids_article,homeandliving_article,avg_shop_cart,total_purchases,avg_cart_itemcount,monday,times_bought_monday,tuesday,times_bought_tuesday,wednesday,times_bought_wednesday,thursday,times_bought_thursday,friday,times_bought_friday,saturday,times_bought_saturday,sunday,times_bought_sunday
0,1,147.01,6,1,0,0,73.505,2,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
1,2,208.34,8,0,0,0,208.340,1,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,3,84.11,0,1,0,0,84.110,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,62.32,1,4,0,0,62.320,1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,89.89,0,0,4,0,44.945,2,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
